In [1]:
# Cell 1: Imports and XIRR helper functions

import math
from datetime import datetime
import tkinter as tk
from tkinter import ttk, messagebox


def xnpv(rate, cashflows, dates):
    """
    NPV for irregular cashflows.
    rate: decimal (e.g., 0.1 for 10%)
    cashflows: list of cash amounts (negative for outflow, positive for inflow)
    dates: list of datetime objects
    """
    if rate <= -1:
        return float('inf')

    t0 = dates[0]
    npv = 0.0
    for cf, d in zip(cashflows, dates):
        days = (d - t0).days
        year_fraction = days / 365.0
        npv += cf / ((1 + rate) ** year_fraction)
    return npv


def xirr(cashflows, dates, tol=1e-6, max_iter=100):
    """
    Find rate r such that NPV = 0 using bisection.
    Returns rate as a decimal (0.12 = 12%)
    """
    low, high = -0.9999, 10.0
    npv_low = xnpv(low, cashflows, dates)
    npv_high = xnpv(high, cashflows, dates)

    if npv_low * npv_high > 0:
        raise ValueError("Cashflows do not produce a sign change in NPV. Check input.")

    for _ in range(max_iter):
        mid = (low + high) / 2
        npv_mid = xnpv(mid, cashflows, dates)

        if abs(npv_mid) < tol:
            return mid

        if npv_low * npv_mid < 0:
            high = mid
            npv_high = npv_mid
        else:
            low = mid
            npv_low = npv_mid

    return (low + high) / 2


print("Cell 1 loaded: imports + XIRR helpers ✅")


Cell 1 loaded: imports + XIRR helpers ✅


In [2]:
# Cell 2: EMI tab function

def create_emi_tab(notebook):
    tab = ttk.Frame(notebook)
    notebook.add(tab, text="EMI")

    # Labels + Entries
    ttk.Label(tab, text="Loan Amount (Principal):").grid(row=0, column=0, sticky="w", padx=5, pady=5)
    principal_entry = ttk.Entry(tab)
    principal_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Annual Interest Rate (%):").grid(row=1, column=0, sticky="w", padx=5, pady=5)
    rate_entry = ttk.Entry(tab)
    rate_entry.grid(row=1, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Tenure (Years):").grid(row=2, column=0, sticky="w", padx=5, pady=5)
    years_entry = ttk.Entry(tab)
    years_entry.grid(row=2, column=1, padx=5, pady=5)

    result_emi = ttk.Label(tab, text="Monthly EMI: ")
    result_total_payment = ttk.Label(tab, text="Total Payment: ")
    result_total_interest = ttk.Label(tab, text="Total Interest: ")

    result_emi.grid(row=4, column=0, columnspan=2, sticky="w", padx=5, pady=5)
    result_total_payment.grid(row=5, column=0, columnspan=2, sticky="w", padx=5, pady=5)
    result_total_interest.grid(row=6, column=0, columnspan=2, sticky="w", padx=5, pady=5)

    def calculate_emi_gui():
        try:
            principal = float(principal_entry.get())
            annual_rate = float(rate_entry.get())
            years = float(years_entry.get())

            monthly_rate = annual_rate / 12 / 100
            n_months = int(years * 12)

            if n_months <= 0:
                messagebox.showerror("Error", "Tenure must be greater than 0.")
                return

            if monthly_rate == 0:
                emi = principal / n_months
            else:
                emi = (principal * monthly_rate * (1 + monthly_rate) ** n_months) / \
                      ((1 + monthly_rate) ** n_months - 1)

            total_payment = emi * n_months
            total_interest = total_payment - principal

            result_emi.config(text=f"Monthly EMI: {emi:,.2f}")
            result_total_payment.config(text=f"Total Payment: {total_payment:,.2f}")
            result_total_interest.config(text=f"Total Interest: {total_interest:,.2f}")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values.")

    calc_button = ttk.Button(tab, text="Calculate EMI", command=calculate_emi_gui)
    calc_button.grid(row=3, column=0, columnspan=2, pady=10)

    return tab


print("Cell 2 loaded: EMI tab ✅")


Cell 2 loaded: EMI tab ✅


In [3]:
# Cell 3: Simple Return tab function

def create_simple_return_tab(notebook):
    tab = ttk.Frame(notebook)
    notebook.add(tab, text="Simple Return")

    ttk.Label(tab, text="Initial Value:").grid(row=0, column=0, sticky="w", padx=5, pady=5)
    initial_entry = ttk.Entry(tab)
    initial_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Final Value:").grid(row=1, column=0, sticky="w", padx=5, pady=5)
    final_entry = ttk.Entry(tab)
    final_entry.grid(row=1, column=1, padx=5, pady=5)

    result_abs = ttk.Label(tab, text="Absolute Return: ")
    result_pct = ttk.Label(tab, text="Return (%): ")

    result_abs.grid(row=3, column=0, columnspan=2, sticky="w", padx=5, pady=5)
    result_pct.grid(row=4, column=0, columnspan=2, sticky="w", padx=5, pady=5)

    def calculate_simple_return_gui():
        try:
            initial = float(initial_entry.get())
            final = float(final_entry.get())

            if initial == 0:
                messagebox.showerror("Error", "Initial value cannot be zero.")
                return

            absolute_return = final - initial
            return_percent = (absolute_return / initial) * 100

            result_abs.config(text=f"Absolute Return: {absolute_return:,.2f}")
            result_pct.config(text=f"Return (%): {return_percent:.2f}%")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values.")

    calc_button = ttk.Button(tab, text="Calculate Return", command=calculate_simple_return_gui)
    calc_button.grid(row=2, column=0, columnspan=2, pady=10)

    return tab


print("Cell 3 loaded: Simple Return tab ✅")


Cell 3 loaded: Simple Return tab ✅


In [4]:
# Cell 4: CAGR tab function

def create_cagr_tab(notebook):
    tab = ttk.Frame(notebook)
    notebook.add(tab, text="CAGR")

    ttk.Label(tab, text="Initial Value:").grid(row=0, column=0, sticky="w", padx=5, pady=5)
    initial_entry = ttk.Entry(tab)
    initial_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Final Value:").grid(row=1, column=0, sticky="w", padx=5, pady=5)
    final_entry = ttk.Entry(tab)
    final_entry.grid(row=1, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Number of Years:").grid(row=2, column=0, sticky="w", padx=5, pady=5)
    years_entry = ttk.Entry(tab)
    years_entry.grid(row=2, column=1, padx=5, pady=5)

    result_cagr = ttk.Label(tab, text="CAGR: ")
    result_cagr.grid(row=4, column=0, columnspan=2, sticky="w", padx=5, pady=5)

    def calculate_cagr_gui():
        try:
            initial = float(initial_entry.get())
            final = float(final_entry.get())
            years = float(years_entry.get())

            if initial <= 0:
                messagebox.showerror("Error", "Initial value must be greater than zero.")
                return
            if years <= 0:
                messagebox.showerror("Error", "Years must be greater than zero.")
                return

            cagr_value = ((final / initial) ** (1 / years) - 1) * 100
            result_cagr.config(text=f"CAGR: {cagr_value:.2f}%")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values.")

    calc_button = ttk.Button(tab, text="Calculate CAGR", command=calculate_cagr_gui)
    calc_button.grid(row=3, column=0, columnspan=2, pady=10)

    return tab


print("Cell 4 loaded: CAGR tab ✅")


Cell 4 loaded: CAGR tab ✅


In [5]:
# Cell 5: SIP Future Value tab function

def create_sip_tab(notebook):
    tab = ttk.Frame(notebook)
    notebook.add(tab, text="SIP FV")

    ttk.Label(tab, text="Monthly SIP Amount:").grid(row=0, column=0, sticky="w", padx=5, pady=5)
    sip_entry = ttk.Entry(tab)
    sip_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Expected Annual Return (%):").grid(row=1, column=0, sticky="w", padx=5, pady=5)
    rate_entry = ttk.Entry(tab)
    rate_entry.grid(row=1, column=1, padx=5, pady=5)

    ttk.Label(tab, text="Duration (Years):").grid(row=2, column=0, sticky="w", padx=5, pady=5)
    years_entry = ttk.Entry(tab)
    years_entry.grid(row=2, column=1, padx=5, pady=5)

    result_invested = ttk.Label(tab, text="Total Invested: ")
    result_fv = ttk.Label(tab, text="Future Value: ")
    result_gain = ttk.Label(tab, text="Wealth Gain: ")

    result_invested.grid(row=4, column=0, columnspan=2, sticky="w", padx=5, pady=5)
    result_fv.grid(row=5, column=0, columnspan=2, sticky="w", padx=5, pady=5)
    result_gain.grid(row=6, column=0, columnspan=2, sticky="w", padx=5, pady=5)

    def calculate_sip_gui():
        try:
            monthly_investment = float(sip_entry.get())
            annual_rate = float(rate_entry.get())
            years = float(years_entry.get())

            monthly_rate = annual_rate / 12 / 100
            n_months = int(years * 12)

            if n_months <= 0:
                messagebox.showerror("Error", "Duration must be greater than 0.")
                return

            if monthly_rate == 0:
                future_value = monthly_investment * n_months
            else:
                future_value = monthly_investment * (((1 + monthly_rate) ** n_months - 1) / monthly_rate) * (1 + monthly_rate)

            total_invested = monthly_investment * n_months
            wealth_gain = future_value - total_invested

            result_invested.config(text=f"Total Invested: {total_invested:,.2f}")
            result_fv.config(text=f"Future Value: {future_value:,.2f}")
            result_gain.config(text=f"Wealth Gain: {wealth_gain:,.2f}")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values.")

    calc_button = ttk.Button(tab, text="Calculate SIP FV", command=calculate_sip_gui)
    calc_button.grid(row=3, column=0, columnspan=2, pady=10)

    return tab


print("Cell 5 loaded: SIP FV tab ✅")


Cell 5 loaded: SIP FV tab ✅


In [6]:
# Cell 6: XIRR tab function

def create_xirr_tab(notebook):
    tab = ttk.Frame(notebook)
    notebook.add(tab, text="XIRR")

    ttk.Label(tab, text="Enter cashflows with dates").grid(row=0, column=0, columnspan=3, padx=5, pady=5, sticky="w")
    ttk.Label(tab, text="(Negative = investment, Positive = return)").grid(row=1, column=0, columnspan=3, padx=5, pady=5, sticky="w")

    ttk.Label(tab, text="Date (YYYY-MM-DD)").grid(row=2, column=0, padx=5, pady=5)
    ttk.Label(tab, text="Amount").grid(row=2, column=1, padx=5, pady=5)

    rows_frame = ttk.Frame(tab)
    rows_frame.grid(row=3, column=0, columnspan=3, padx=5, pady=5, sticky="w")

    cashflow_rows = []

    def add_row(date_default="", amount_default=""):
        row_index = len(cashflow_rows)
        date_entry = ttk.Entry(rows_frame)
        amount_entry = ttk.Entry(rows_frame)

        date_entry.grid(row=row_index, column=0, padx=5, pady=2)
        amount_entry.grid(row=row_index, column=1, padx=5, pady=2)

        date_entry.insert(0, date_default)
        amount_entry.insert(0, amount_default)

        cashflow_rows.append((date_entry, amount_entry))

    # Add two default rows
    add_row()
    add_row()

    def calculate_xirr_gui():
        try:
            cashflows = []
            dates = []

            for date_entry, amount_entry in cashflow_rows:
                date_str = date_entry.get().strip()
                amt_str = amount_entry.get().strip()

                if not date_str and not amt_str:
                    continue  # skip empty rows

                if not date_str or not amt_str:
                    messagebox.showerror("Error", "Each used row must have both date and amount.")
                    return

                try:
                    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                except ValueError:
                    messagebox.showerror("Error", f"Invalid date format: {date_str}. Use YYYY-MM-DD.")
                    return

                try:
                    amt = float(amt_str)
                except ValueError:
                    messagebox.showerror("Error", f"Invalid amount: {amt_str}.")
                    return

                dates.append(date_obj)
                cashflows.append(amt)

            if len(cashflows) < 2:
                messagebox.showerror("Error", "At least two cashflows are required.")
                return

            rate = xirr(cashflows, dates)
            rate_percent = rate * 100
            result_label.config(text=f"XIRR: {rate_percent:.2f}%")

        except ValueError as e:
            messagebox.showerror("Error", str(e))
        except Exception as e:
            messagebox.showerror("Error", f"Unexpected error: {e}")

    add_button = ttk.Button(tab, text="Add Cashflow Row", command=add_row)
    add_button.grid(row=4, column=0, padx=5, pady=10, sticky="w")

    calc_button = ttk.Button(tab, text="Calculate XIRR", command=calculate_xirr_gui)
    calc_button.grid(row=4, column=1, padx=5, pady=10, sticky="w")

    result_label = ttk.Label(tab, text="XIRR: ")
    result_label.grid(row=5, column=0, columnspan=3, padx=5, pady=10, sticky="w")

    return tab


print("Cell 6 loaded: XIRR tab ✅")


Cell 6 loaded: XIRR tab ✅


In [7]:
# Cell 7: Main app - create window, tabs, and start Tkinter loop

def main():
    root = tk.Tk()
    root.title("Financial Calculator (EMI, Return, CAGR, SIP, XIRR)")
    root.geometry("600x400")

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill="both")

    # Add all tabs
    create_emi_tab(notebook)
    create_simple_return_tab(notebook)
    create_cagr_tab(notebook)
    create_sip_tab(notebook)
    create_xirr_tab(notebook)

    root.mainloop()


print("Cell 7 loaded: main() defined ✅")


Cell 7 loaded: main() defined ✅


In [8]:
# Cell 8: Run the GUI

main()
